In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

ds = load_dataset("openai/summarize_from_feedback", "comparisons")

0000.parquet:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/22.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/92858 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/86086 [00:00<?, ? examples/s]

In [3]:
from datasets import load_dataset

axis = load_dataset("openai/summarize_from_feedback", "axis")

In [4]:
train_df_compar = pd.DataFrame(ds["train"])
validation_df_compar = pd.DataFrame(ds["validation"])

test_df_axis = pd.DataFrame(axis["test"])
validation_df_axis = pd.DataFrame(axis["validation"])


In [5]:
validation_df_compar['confidence'] = validation_df_compar['extra'].apply(lambda x: x['confidence'])
train_df_compar['confidence'] = train_df_compar['extra'].apply(lambda x: x['confidence'])

validation_df_compar['ID'] = validation_df_compar['info'].apply(lambda x: x['id'])
train_df_compar['ID'] = train_df_compar['info'].apply(lambda x: x['id'])

### Creating the clicks and Skips

In [6]:
clicks_compar = validation_df_compar[validation_df_compar['confidence'].fillna(0) >= 6]
skips_compar = validation_df_compar[validation_df_compar['confidence'].fillna(0) < 6]

clicks_compar_train = train_df_compar[train_df_compar['confidence'].fillna(0) >= 6]
skips_compar_train = train_df_compar[train_df_compar['confidence'].fillna(0) < 6]


##### Clicks if confidence>= 6 else Skips

In [9]:
clicks_final_df = pd.concat([clicks_compar_train, clicks_compar], ignore_index=True)
skips_final_df = pd.concat([skips_compar_train, skips_compar], ignore_index=True)

In [10]:
clicks_final_df['post'] = clicks_final_df['info'].apply(lambda x: x['post'])
clicks_final_df['title'] = clicks_final_df['info'].apply(lambda x: x['title'])
clicks_final_df['subreddit'] = clicks_final_df['info'].apply(lambda x: x['subreddit'])


skips_final_df['post'] = skips_final_df['info'].apply(lambda x: x['post'])
skips_final_df['title'] = skips_final_df['info'].apply(lambda x: x['title'])
skips_final_df['subreddit'] = skips_final_df['info'].apply(lambda x: x['subreddit'])

### Filtering out data as we are only working with the reddit posts and data --> removing the cnndm batches of data

In [11]:
df_filtered_clicks = clicks_final_df[clicks_final_df["batch"] != "cnndm2"]
df_filtered_clicks= df_filtered_clicks [clicks_final_df["batch"] != "cnndm0"]
df_filtered_clicks = df_filtered_clicks [clicks_final_df["batch"] != "batch0_cnndm"]

df_filtered_skips = skips_final_df[skips_final_df["batch"] != "cnndm2"]
df_filtered_skips= df_filtered_skips[skips_final_df["batch"] != "cnndm0"]
df_filtered_skips = df_filtered_skips[skips_final_df["batch"] != "batch0_cnndm"]

/tmp/ipykernel_1944/2042986461.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtered_clicks= df_filtered_clicks [clicks_final_df["batch"] != "cnndm0"]
/tmp/ipykernel_1944/2042986461.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtered_clicks = df_filtered_clicks [clicks_final_df["batch"] != "batch0_cnndm"]
/tmp/ipykernel_1944/2042986461.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtered_skips= df_filtered_skips[skips_final_df["batch"] != "cnndm0"]
/tmp/ipykernel_1944/2042986461.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtered_skips = df_filtered_skips[skips_final_df["batch"] != "batch0_cnndm"]


In [18]:
df_filtered_clicks['summaries'][0]

[{'text': " My name wasn't on the list for a lab. As such someone found out and is using it to access other students' homework assignments. Nobody in class will know.",
  'policy': 'sup4_ppo_rm3_kl10',
  'note': "OP's lab account password is being used by someone else."},
 {'text': ' A math class had us sign up for a lab group by providing our name and student ID numbers. Someone discovered how easy it was to steal homework assignments and nobody will know.',
  'policy': 'sup4_ppo_rm3_kl10',
  'note': 'ok'}]

In [28]:
df_filtered_skips['Selected Summary'] = df_filtered_skips.apply(
    lambda row: row['summaries'][row['choice']]['text'], axis=1
)

df_filtered_clicks['Selected Summary'] = df_filtered_clicks.apply(
    lambda row: row['summaries'][row['choice']]['text'], axis=1
)

In [29]:
df_filtered_clicks["Action"] = "click"
df_filtered_skips["Action"] = "skip"

In [30]:
final_df = pd.concat([df_filtered_clicks, df_filtered_skips], ignore_index=True) #final combined df of both clicks and skips

In [31]:
final_df

,info,summaries,choice,worker,batch,split,extra,confidence,ID,post,title,subreddit,Action,Selected Summary
0,"{'id': 't3_f2bvu', 'post': 'A little backgroun...",[{'text': ' My name wasn't on the list for a l...,1,M4bdOszgybjO2qg2Dth5I1GOYAvE7V,batch10,train,{'confidence': 8},8.0,t3_f2bvu,A little background first. Like most engineers...,Averted a shit storm in my math class today an...,AskReddit,click,A math class had us sign up for a lab group b...
1,"{'id': 't3_f2bvu', 'post': 'A little backgroun...",[{'text': ' A math class had us sign up for a ...,1,M4bdOszgybjO2qg2Dth5I1GOYAvE7V,batch10,train,{'confidence': 9},9.0,t3_f2bvu,A little background first. Like most engineers...,Averted a shit storm in my math class today an...,AskReddit,click,I accidentally discovered that my university ...
2,"{'id': 't3_f2bvu', 'post': 'A little backgroun...","[{'text': ' Unbeknownst to me, my self assigne...",1,M4bdOszgybjO2qg2Dth5I1GOYAvE7V,batch10,train,{'confidence': 6},6.0,t3_f2bvu,A little background first. Like most engineers...,Averted a shit storm in my math class today an...,AskReddit,click,My name wasn't on the list for a lab. As such...
3,"{'id': 't3_f2bvu', 'post': 'A little backgroun...",[{'text': ' My name wasn't on the list for a l...,1,M4bdOszgybjO2qg2Dth5I1GOYAvE7V,batch10,train,{'confidence': 9},9.0,t3_f2bvu,A little background first. Like most engineers...,Averted a shit storm in my math class today an...,AskReddit,click,I accidentally discovered that my university ...
4,"{'id': 't3_f2bvu', 'post': 'A little backgroun...","[{'text': ' Unbeknownst to me, my self assigne...",1,M4bdOszgybjO2qg2Dth5I1GOYAvE7V,batch10,train,{'confidence': 9},9.0,t3_f2bvu,A little background first. Like most engineers...,Averted a shit storm in my math class today an...,AskReddit,click,I accidentally discovered that my university ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176655,"{'id': 't3_kz1gv', 'post': 'Hello World of Red...",[{'text': ' I'm a 21year old female anemic tak...,0,RMwrIV50cNusBthNvLs1wSNdqFpQAg,batch9,valid1,{'confidence': 1},1.0,t3_kz1gv,Hello World of Reddit\n\nI'm a 21 year old fem...,"I have iron deficient anemia, need help and/or...",AskReddit,skip,I'm a 21year old female anemic taking iron su...
176656,"{'id': None, 'post': 'I don't know how to ask ...","[{'text': 'Boyfriend talks very slowly, I feel...",0,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,valid2,{'confidence': 2},2.0,None,I don't know how to ask this without it soundi...,Me [27 F] with my boyfriend [24 M] for four mo...,r/relationships,skip,"Boyfriend talks very slowly, I feel frustrated..."
176657,"{'id': None, 'post': 'I honestly do not care a...",[{'text': 'Medical field looks like its going ...,0,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,valid1,{'confidence': 2},2.0,None,I honestly do not care anymore. \n\nExcluding ...,Almost every career in the medical field is sa...,r/Advice,skip,Medical field looks like its going to hell. Wh...
176658,"{'id': None, 'post': 'Someone I consider to be...","[{'text': 'Met a guy on reddit, became best fr...",0,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,valid2,{'confidence': 2},2.0,None,Someone I consider to be a best friend and I a...,I'm [26/f] in US and my best friend [32/m] is ...,r/relationship_advice,skip,"Met a guy on reddit, became best friends, met,..."


#### Creating the replica of news.tsv that exists in PENS for openai-reddit

In [32]:
summary_map = {}
summary_id_counter = 0
summary_id_map = []
summary_texts = []

for index, row in final_df.iterrows():
    summary_text = row['Selected Summary']
    if summary_text not in summary_map:
        summary_id_counter += 1
        summary_map[summary_text] = f"S{summary_id_counter}"
    summary_id_map.append(summary_map[summary_text])
    summary_texts.append(summary_text)

news_openai= pd.DataFrame({
    'newsID': final_df['info'].apply(lambda x: x['id'] if x else None),
    'userID': final_df['worker'],
    'summaryID': summary_id_map,
    'summary': summary_texts
})

news_openai


,newsID,userID,summaryID,summary
0,t3_f2bvu,M4bdOszgybjO2qg2Dth5I1GOYAvE7V,S1,A math class had us sign up for a lab group b...
1,t3_f2bvu,M4bdOszgybjO2qg2Dth5I1GOYAvE7V,S2,I accidentally discovered that my university ...
2,t3_f2bvu,M4bdOszgybjO2qg2Dth5I1GOYAvE7V,S3,My name wasn't on the list for a lab. As such...
3,t3_f2bvu,M4bdOszgybjO2qg2Dth5I1GOYAvE7V,S2,I accidentally discovered that my university ...
4,t3_f2bvu,M4bdOszgybjO2qg2Dth5I1GOYAvE7V,S2,I accidentally discovered that my university ...
...,...,...,...,...
176655,t3_kz1gv,RMwrIV50cNusBthNvLs1wSNdqFpQAg,S31175,I'm a 21year old female anemic taking iron su...
176656,None,soP7Okyp0e1WdYxQC9IldpwnznT6V1,S74662,"Boyfriend talks very slowly, I feel frustrated..."
176657,None,soP7Okyp0e1WdYxQC9IldpwnznT6V1,S74663,Medical field looks like its going to hell. Wh...
176658,None,soP7Okyp0e1WdYxQC9IldpwnznT6V1,S74664,"Met a guy on reddit, became best friends, met,..."


In [33]:
### saving the news_openai , clicks and skips